In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import random
import math
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Input, Dense, Dropout, Flatten, Activation, Conv1D, Add, MaxPooling1D, BatchNormalization
from tensorflow.keras.utils import to_categorical

def PreProcess(FileName):
    #Function to preprocess data, will split proteins into individual list elements
    #and eliminate any much too long or much too short sequences
    #Requiers a filename of type TXT in the format "Name.txt"
    #Outputs the meta list, the sequence list and a count of proteins in them

    FilePath = 'drive/My Drive/Project/'+FileName
    file = open(FilePath,"r")
    #Sets an active file from which to read
    #And sets it to read mode (rather than write mode)
    RawFile = file.readlines()
    #Reads each line to a new list element
    Sequence = []
    Meta = []
    #initialise empty lists to contain the sequence of amino acids and the 
    #metadata of each sequence (Actual function etc)
    Count = -1
    #Initialises index (count) for the lists, to be able to access specific elements and
    #check the whole dataset is accessed.
    for row in RawFile:
        #For each row in the non binding amino acid sequence dataset
        if row[0] == ">":
            #if the row starts with a >, it contains the metadata and a string of text
            #so this row is stored seperatly and marks the beginning of a new protei
            Count+=1
            #increment count
            Sequence.append("")
            #add an element to the list for the new protein
            Meta.append(row)
            #and append the meta with the metadata
        else:
            row = row.replace("\n", "")#Remove all newline characters from the text string
            Sequence[Count] = Sequence[Count] + row
            #else, the row is sequence data and should be appended to the current running sequence
    
    length = []#Create a list to contain the length of the sequence list
    FinalMeta = []#The final sanitised output for MetaData (Unused but futureproof)
    FinalSequence = []#List to contain the final sequences after any too long or short are removed
    FinalLength = []#List containing the lengths of all finalSequence sequences (Essentially a parallel array, 
    #would work in a 2D list but would need to be divided ever time its used
    
    MaxLength = 1000 #The max sequence length. Anything larger would be anomalous
    MinLength = 50 # Same for the min length
    Count = -1 #An index count
    for each in Sequence: # For ever item in sequence, add the length of that item to the lengths list
        length.append(len(each))
    for each in length:#for every item in length, if the associated sequence item is valid, add it to finalSequence
    #And finalMeta etc. otherwisse just move on
        Count+=1
        if each<=MaxLength and each>=MinLength:
            FinalMeta.append(Meta[Count])
            FinalSequence.append(Sequence[Count])
            FinalLength.append(length[Count])
    #Return FinalMeta, data on each sequence in case it's useful later, final sequence and finallength        
    return FinalMeta,FinalSequence,FinalLength

def ConvertToInt(AASequences):
    #Must be parsed a list of Raw Amino Acid sequences, and replaces the letter codes with numbers
    #Then returns the changed list
    letterCodes = {'A':1, 'R':2, 'N':3, 'D':4, 'C':5, 'Q':6, 'E':7, 'G':8, 'H':9, 'I':10,
     'L':11, 'K':12, 'M':13, 'F':14, 'P':15, 'S':16, 'T':17, 'W':18, 'Y':19, 'V':20, 'U':21,
     'O':22}
    #Dictionary of Letter codes to convert, each represents a numeric value
    #Alanine = A, Arginine = R etc.
    #The addition of the 21st,
    #Selenocysteine = U came later
    #Pyrrolysine = O is 22
    #Other non essential amino acids are not included such as Aspartic Acid andGlutamic Acid
    #If they come up add them to the dictionary. otherwise they are too rare to conside
    #Especially with exclusivly human Amino Acids/Proteins
    intAll = []#List of lists for the integer converted sequences
    intProtein = []#list for each protein represented as integers
    for Sequence in AASequences:
      #for each sequence
        for each in Sequence:
          #for each letter
            intProtein.append(letterCodes[each])
            #lookup in dictionary the conversion, and add the conversion to the proteinList
        intAll.append(intProtein)
        #add the entier protein to the All list
        intProtein = []
    return intAll # return the list of lists of amino acid conversions

def Pad(Binding, NonBinding, PadToLength):
    #Pads Each amino Acid sequence to a regular length to be parsed into the CNN properly
    #PAds to the largest length, in this case 1000
    for i in range(len(Binding)):
        #For each sequence
        while len(Binding[i])< PadToLength:
        #While its too short
            Binding[i].append(0)
            #Add the value "0" to the end
    for i in range(len(NonBinding)):
        while len(NonBinding[i]) < PadToLength:
            NonBinding[i].append(0)
    print("Padding Completed")#Update terminal
    return Binding, NonBinding#Output Padded lists

def Split(Binding, NonBinding):
    #Converts the binding and non binding into X and Y train, Val, and test sets.
    #and returns all 6 of thse sets XTrain, YTrain, XVal, YVal etc
    BindingArray = np.array(Binding) #Convert List to Array
    BindingLabelsArray = np.ones(len(Binding))#Create an array of ones for the Y data (1 is the boolean for Bunding=True)
    #and length equal to the amount of labels needed
    BindingArray.astype(int)#Converts the Array to an array of integers rather than floats or strings
    
    NonBindingArray = np.array(NonBinding)#Convert list to array
    NonBindingLabelsArray = np.zeros(len(NonBinding))#Create an array of Zeroes for the boolean for non binding for Y Data
    NonBindingArray.astype(int)#Converts the arrat to integer
    
    WholeDataset = np.concatenate((BindingArray,NonBindingArray), axis=0)#Concatenate both arrays to be shuffled (XData)
    WholeLabels = np.concatenate((BindingLabelsArray,NonBindingLabelsArray), axis=0)#concatenate labelarrays (YData)
    
    index = np.arange(len(WholeLabels))#Creates an index to shuffle both arryas the same way, to keep X and Y associated
    #The Index array is created with the values 1,2,3,4,5,6,7,8... to the length needed.
    np.random.shuffle(index)#Shuffles the index array randomly
    
    WholeDataset = WholeDataset[index]#Shuffles the XData to matsh the shuffled array
    WholeLabels = WholeLabels[index]#Shuffles the YData to match the shuffled array
    
    TrainDivision = round(len(WholeLabels) * 0.6)#Creates indexes where the data should be divided into Train
    ValidateDivision = round(len(WholeLabels) * 0.9)#Creates a second divide for Validation and test
    #80% Train, 10% Validate, 10% Test 
    
    #Actual division of the data using array indexing 0 to divide, divide to divide, divide to end.
    Train = WholeDataset[:TrainDivision]#XTrain
    TrainLabels = WholeLabels[:TrainDivision]#YTrain
    Validate = WholeDataset[TrainDivision:ValidateDivision]#XValidate
    ValidateLabels = WholeLabels[TrainDivision:ValidateDivision]#YValidate
    Test = WholeDataset[ValidateDivision:]#XTest
    TestLabels = WholeLabels[ValidateDivision:]#YTest
    
    return (Train,TrainLabels,Validate,ValidateLabels,Test,TestLabels)#Returns the 6 arrays of data

def EvaluateModel(Predictions):
    TruePositive = 0
    TrueNegative = 0
    FalsePositive = 0
    FalseNegative = 0

    for i in range(len(Predictions)):
      #print("Predicted: ", round(Predictions[i][0]), " Actual: ", TestY[i])
      if (TestY[i][0] == 1 and round(Predictions[i][0])) == 1:
        #True positive
        TruePositive+=1
      if TestY[i][0] == 0 and round(Predictions[i][0]) == 0:
        #True Negative
        #Predicted
        TrueNegative+=1
      if TestY[i][0] == 0 and round(Predictions[i][0]) == 1:
        #False Positive
        FalsePositive+=1
      if TestY[i][0] == 1 and round(Predictions[i][0]) == 0:
        #False Negative
        FalseNegative+=1
      

    print("True Positive = ", TruePositive)
    print("True Negative = ", TrueNegative)
    print("False Positive = ", FalsePositive)
    print("False Negative = ", FalseNegative)

    Accuracy = (TruePositive + TrueNegative) / (TruePositive + TrueNegative + FalsePositive + FalseNegative)
    Specificity = TrueNegative / (TrueNegative+FalsePositive)
    Precision = TruePositive / (TruePositive+FalsePositive)
    Recall = TruePositive / (TruePositive+FalseNegative)

    print("Accuracy = ", Accuracy*100,"%")
    print("Specificity = ", Specificity*100,"%")
    print("Precision = ", Precision*100,"%")
    print("Recall = ", Recall*100,"%")
    return Accuracy*100, Precision*100, Specificity*100, Recall*100

NonBindingMeta,NonBindingSequences,NonBindingCount = PreProcess("NotDNABinding.txt")
#CAlls the PreProcess function to read the data from the given file and return the three relevant lists
BindingMeta,BindingSequences,BindingCount = PreProcess("DNABinding.txt")
#Calls PreProcess to read data for binding Proteins and store them in three lists too

NonBindingSequencesInt = ConvertToInt(NonBindingSequences)
BindingSequencesInt = ConvertToInt(BindingSequences)
#Calls the ConvertToInt on the sequences to convert letter codes to integer values

if max(BindingCount) > max(NonBindingCount):
    #Only calls the function with the longest max length so they all get padded the same
    BindingSequencesInt, NonBindingSequencesInt = Pad(BindingSequencesInt, NonBindingSequencesInt, max(BindingCount))
else:
    BindingSequencesInt, NonBindingSequencesInt = Pad(BindingSequencesInt, NonBindingSequencesInt, max(NonBindingCount))
#Padds the integer sequences to even length 1000 or less

TrainData,TrainLabels,ValData,ValLabels,TestData,TestLabels = Split(BindingSequencesInt, NonBindingSequencesInt)
#Calls the Split function to split the data int X and Y Train, Validate, and Test
print("Done")#Update console of data being ready
#0 is non binding 1 is binding

#Creating the model
#There are three kinds of Layers: Convolutional, Pooling and fully connected(or dense)
#Also the ReLU or Rectified Linear Unit layer. which houses the rectified linear activation function used for backpropogation of errors
#ReLU is a type of hidden layer, part of the dense or right before fully connected
#In My case I will begin with a Convolution layer and end with a dense fully connected layer
#the main body of the CNN will consist of one round of Batch Normalisation, then one of ReLU and an addition layer
#Input > Conv1D > Batch Norm > ReLU > Conv1D > Pooling > Dropout > Flatten > Dense
#Then Various other combinations will be attampted i.e.
#Input > Conv1D > Batch Norm > ReLU > Conv1D > Batch Norm > ReLU > Conv1D> Batch Norm > ReLU > Conv1D > Add > Pooling > Dropout > Flatten > Dense

TrainX = to_categorical(TrainData)
ValX = to_categorical(ValData)
TestX = to_categorical(TestData)

TrainY = to_categorical(TrainLabels)
ValY = to_categorical(ValLabels)
TestY = to_categorical(TestLabels)

print(TrainX.shape,ValX.shape,TestX.shape)
print(TrainY.shape,ValY.shape,TestY.shape)


#Model1Input = Input(shape = (1000,22))#The shape of the input data, each AA chain is 1000x1 long with padding
#Model1Conv1 = Conv1D(50, 1, padding = 'same')(Model1Input)#Adds a second layer to the CNN, the 1D convolution layer
#Model1Batch1 = BatchNormalization()(Model1Conv1)#Adds a BAtchNormalisation Layer
#Model1ReLU = Activation('relu')(Model1Batch1)#Adds a ReLU activation function layer for backpropegation of errors and adjustment for later epochs
#Model1Conv2 = Conv1D(50, 1, padding = 'same', dilation_rate = 2, kernel_regularizer=l2(0.001))(Model1ReLU)#Second Convolutional Layer
#Model1Pool = MaxPooling1D(3)(Model1Conv2)#Pooling Layer
#Model1Dropout = Dropout(0.5)(Model1Pool)#Dropout Layer
#Model1Flatten = Flatten()(Model1Dropout)#Flatten the result of the previous layers to be parsed to the dense layer
#Model1Dense = Dense(2, activation = 'softmax', kernel_regularizer=l2(0.0001))(Model1Flatten)# D E N S E Layer. The main part of the CNN

UndoneFilters = [45,50,55,60,65,70,75,80,85,90,95,100]
filters = [5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,95,100]
batchSizes = [32,64,96,128,160,192,224,256,288,320,352,384,416,448,480,512]
#The best Epochs is 1500

for fil in UndoneFilters:
  print(fil)
  ModelFirstInput = Input(shape = (1000,22))
  ModelFirstConv = Conv1D(fil,1, padding = 'same')(ModelFirstInput)
  ModelFirstPool = MaxPooling1D(3)(ModelFirstConv)
  ModelFirstDropout = Dropout(0.5)(ModelFirstPool)
  ModelFirstFlatten = Flatten()(ModelFirstDropout)
  ModelFirstDense = Dense(2, activation = 'softmax', kernel_regularizer=l2(0.0001))(ModelFirstFlatten)
  #Model1 = Model(inputs = Model1Input, outputs = Model1Dense)
  ModelFirst = Model(inputs = ModelFirstInput, outputs = ModelFirstDense)

  ModelFirst.compile(optimizer = 'adam', loss='categorical_crossentropy', metrics=['accuracy'])
  ModelFirst.summary()
  FittedModelFirst = ModelFirst.fit(TrainX,TrainY,epochs=1500,batch_size=256,validation_data=(ValX,ValY))
  
  filepath = "drive/My Drive/Project/Models/Model1_1500_fil_"+str(fil)+".h5"
  ModelFirst.save(filepath)


#Solidifes the model as a model to be called and used from the first and last layers,
#Each layer parses to the next layer so has to have a readable output by the next layer
#The final layer categorises the data properly

#Model1.compile(optimizer = 'adam', loss='categorical_crossentropy', metrics=['accuracy'])

#Compiles the Model

EarlyCallback = EarlyStopping(monitor='val_loss',patience=3,verbose=1)
#Model1.summary()#Shows a summary of each layer, outputs, expecte inputs and parameters. Also the unique name



#FittedModel1 = Model1.fit(TrainX,TrainY,epochs=2500,batch_size=256,validation_data=(ValX,ValY))#,callbacks=[EarlyCallback])

#filepath = "drive/My Drive/Project/Models/Model1_Final"+str(epoch)+".h5"
#Model1.save(filepath)
#filepath = "drive/My Drive/Project/Models/Model1_1250.h5"
#Model1.save(filepath)

#Predictions = Model1.predict_on_batch(TestX)
#epochAccuracy, epochPrecision, epochSpecificity, epochRecall = EvaluateModel(Predictions)
#accuracies.append(epochAccuracy)
#precisions.append(epochPrecision)
#specificities.append(epochSpecificity)
#recalls.append(epochRecall) 

#filepath = "drive/My Drive/Project/Models/ModelFirst1250.h5"
#ModelFirst = load_model(filepath)

M_accuracies = []
M_precisions = []
M_specificities = []
M_recalls = []

for fil in filters:
  filepath = "drive/My Drive/Project/Models/Model1_1500_fil_"+str(fil)+".h5"
  ModelFirst = load_model(filepath)
  Predictions = ModelFirst.predict_on_batch(TestX) 
  epochAccuracy, epochPrecision, epochSpecificity, epochRecall = EvaluateModel(Predictions)
  M_accuracies.append(epochAccuracy)
  M_precisions.append(epochPrecision)
  M_specificities.append(epochSpecificity)
  M_recalls.append(epochRecall)

print(M_accuracies)
print(M_precisions) 
print(M_specificities) 
print(M_recalls)
#Fitting the model to the Train and Validate Data. It's given the earlyCallback Callback which monitors 'loss' 
  
  #It will return an display rate
  #filepath = "drive/My Drive/Project/Models/Model1_"+str(epoch)+".h5"
  #Model1 = load_model(filepath)
  #Model1.save(filepath)
  #new_model = load_model(filepath)



Padding Completed
Done
(10728, 1000, 22) (5364, 1000, 22) (1788, 1000, 22)
(10728, 2) (5364, 2) (1788, 2)
45
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1000, 22)]        0         
_________________________________________________________________
conv1d (Conv1D)              (None, 1000, 45)          1035      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 333, 45)           0         
_________________________________________________________________
dropout (Dropout)            (None, 333, 45)           0         
_________________________________________________________________
flatten (Flatten)            (None, 14985)             0         
_________________________________________________________________
dense (Dense)                (None, 2)                 29972     
Total params: 31,0